In [1]:
import PIL
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import keras

from keras import layers
from keras import backend as K
from keras.models import Model
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Conv1D, Flatten
from keras.layers import CuDNNLSTM, GRU, LSTM, ConvLSTM2D, CuDNNGRU
from keras.layers.normalization import BatchNormalization

from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

Using TensorFlow backend.


In [2]:
image = Image.open("circle.png")

In [3]:
print(image.size)
print(image.getpixel((1,1)))
print(image.getpixel((500,500)))

(913, 824)
(255, 255, 255)
(255, 127, 39)


In [4]:
# generate data

rows_list = list()
noise = list()
noises = 15
for i in range(10000):
    rand_x = np.random.randint(image.size[0])
    rand_y = np.random.randint(image.size[1])
    noise_dict = dict()
    for n in range(noises):
        noise_dict["noise%s"%n] = np.random.randint(image.size[np.random.randint(len(image.size))])
    if image.getpixel((rand_x, rand_y)) == (255, 127, 39):  # ms orange lul
        noise_dict.update({"x":rand_x, "y":rand_y, "target":1})
    else:
        noise_dict.update({"x":rand_x, "y":rand_y, "target":0})
    rows_list.append(noise_dict)

df = pd.DataFrame(rows_list)
df.head(20)

,noise0,noise1,noise10,noise11,noise12,noise13,noise14,noise2,noise3,noise4,noise5,noise6,noise7,noise8,noise9,target,x,y
0,869,384,231,816,533,738,725,743,587,151,128,456,777,722,527,0,672,163
1,233,267,191,793,297,183,804,767,697,616,255,254,552,80,65,1,478,424
2,319,257,224,569,838,260,745,685,519,293,523,909,703,319,338,0,254,517
3,96,592,111,593,583,732,161,281,109,66,356,821,85,536,700,1,593,320
4,244,162,733,653,654,42,268,769,31,267,529,665,621,276,669,0,92,424
5,374,380,6,439,396,167,625,176,879,642,635,613,796,216,713,0,650,821
6,405,355,261,499,664,649,792,908,892,798,287,132,290,89,258,0,887,675
7,798,207,725,308,601,332,566,463,317,43,273,391,514,226,381,1,359,375
8,514,318,786,0,787,538,244,309,383,798,232,643,142,203,791,1,309,359
9,280,192,507,328,675,766,76,489,89,492,675,648,79,195,477,0,53,304


In [5]:
def preprocess(data, validation_data=None):
    data = data.convert_objects(convert_numeric=True)
    data = data.fillna(0)
    
    train_in = data.drop(columns=["target"])
    train_out = data["target"]
    
    traini_np = train_in.values
    
    scaler = preprocessing.MinMaxScaler()
    train_in = scaler.fit_transform(traini_np)
    
    if validation_data is None:
        return train_in, train_out
    else:
        validation_data = validation_data.convert_objects(convert_numeric=True)
        validation_data = validation_data.fillna(0)
        
        val_np = validation_data.values
        val = scaler.transform(val_np)
        return train_in, train_out, val

In [6]:
features, target = preprocess(df)
t_in, v_in, t_out, v_out = train_test_split(features, target, test_size = 0.2, random_state=0)

c:\python37\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [7]:
rf = RandomForestClassifier(n_estimators = 100, random_state=0, n_jobs=4, verbose=1)
rf.fit(t_in, t_out)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                       oob_score=False, random_state=0, verbose=1,
                       warm_start=False)

In [8]:
def acc_printer(pred, actual):
    assert len(pred) == len(actual)
    fp=0;tp=0;fn=0;tn=0
    for i in range(len(pred)):
        if pred[i]==actual[i]==1:
            tp+=1
        elif pred[i]==actual[i]==0:
            tn+=1
        elif 1==pred[i]!=actual[i]:
            fp+=1
        elif 0==pred[i]!=actual[i]:
            fn+=1
        else:
            raise Exception("wtf")
    print("Stats:\nCorrect true: %s\nCorrect false: %s\nFalse positive: %s\nFalse negative: %s\n" % (tp, tn, fp, fn))
    
def add_useless_dimension(in_array):
    new_shape = list(np.shape(in_array))
    new_shape.append(1)
    in_array = np.array(in_array)
    return in_array.reshape(tuple(new_shape))

In [51]:
sols = rf.predict(v_in)
acc_printer(sols, v_out.values)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


Stats:
Correct true: 338
Correct false: 1625
False positive: 27
False negative: 10



In [49]:
def cnn_gru_model(data):
    if len(np.shape(data["x_train"])) < 3:
        data["x_train"] = add_useless_dimension(data["x_train"])
    input_x = keras.Input(shape=(np.shape(data['x_train'])[1], 1), name="x_input")
    print(input_x)
    
    conv = (Conv1D(64, 1, activation='relu'))(input_x)
    conv = BatchNormalization()(conv)
    
    gru = (GRU(64, return_sequences=True))(conv)
    gru = (GRU(32, return_sequences=True))(gru)
    gru = (GRU(16, return_sequences=False))(gru)
    
    gru = layers.Dense(64, activation='sigmoid')(gru)
    gru = layers.Dense(32, activation='sigmoid')(gru)
    y = layers.Dense(1, activation='sigmoid')(gru)
    
    #model
    #classifier = Model([input_x, input_xd], y)

    classifier = Model([input_x], y)
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

def cnn_lstm_model(data):
    if len(np.shape(data["x_train"])) < 3:
        data["x_train"] = add_useless_dimension(data["x_train"])
    input_x = keras.Input(shape=(np.shape(data['x_train'])[1], 1), name="x_input")
    
    conv = (Conv1D(64, 1, activation='relu'))(input_x)
    conv = BatchNormalization()(conv)
    conv = (Conv1D(64, 2, activation='relu'))(conv)
    conv = BatchNormalization()(conv)
    conv = (Conv1D(64, 3, activation='relu'))(conv)
    conv = BatchNormalization()(conv)
            
    lstm = (LSTM(64, return_sequences=True))(conv)
    lstm = (LSTM(64, return_sequences=True))(lstm)
    lstm = (LSTM(64, return_sequences=False))(lstm)

    #output layer
    lstm = layers.Dense(64, activation='relu')(lstm)
    lstm = layers.Dense(32, activation='relu')(lstm)
    y = layers.Dense(1, activation='sigmoid')(lstm)
    
    classifier = Model([input_x], y)
    classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

def train_model(model, t_in, t_out, v_in, v_out, MODEL_NAME, class_weight=None):
    if len(np.shape(t_in)) < 3:
        t_in = add_useless_dimension(t_in)
        v_in = add_useless_dimension(v_in)
    callbacks = [EarlyStopping(monitor='val_loss', patience=15),
             ModelCheckpoint(filepath="models/"+MODEL_NAME+".h5", monitor='val_loss', verbose=2, save_best_only=True)]
    model.summary()
    model.fit(x={"x_input": t_in}, y=t_out, shuffle=True, epochs=100, callbacks=callbacks, batch_size=64,
              verbose=1, validation_data=([v_in], v_out), class_weight=class_weight)
    return model

print(np.shape(t_in))
print(np.shape(t_out))
print(np.shape(v_in))
print(np.shape(v_out))
network = cnn_gru_model({'x_train': t_in})
model = train_model(network, t_in, t_out, v_in, v_out, "lul")


(8000, 17)
(8000,)
(2000, 17)
(2000,)
Tensor("x_input_24:0", shape=(None, 17, 1), dtype=float32)
Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x_input (InputLayer)         (None, 17, 1)             0         
_________________________________________________________________
conv1d_73 (Conv1D)           (None, 17, 64)            128       
_________________________________________________________________
batch_normalization_73 (Batc (None, 17, 64)            256       
_________________________________________________________________
gru_1 (GRU)                  (None, 17, 64)            24768     
_________________________________________________________________
gru_2 (GRU)                  (None, 17, 32)            9312      
_________________________________________________________________
gru_3 (GRU)                  (None, 16)                2352      
___________________________

8000/8000 [==============================] - 4s 503us/step - loss: 0.0362 - accuracy: 0.9849 - val_loss: 0.0332 - val_accuracy: 0.9850

Epoch 00032: val_loss did not improve from 0.02183


In [50]:
sols = model.predict(add_useless_dimension(v_in))
acc_printer(np.round(sols), v_out.values)

Stats:
Correct true: 342
Correct false: 1628
False positive: 24
False negative: 6



In [22]:
keras.utils.np_utils.to_categorical([1,1,0,1])

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)